# Домашнее задание № 3. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [ ]:
!pip install razdel

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import numpy as np
from collections import Counter
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
import random

In [ ]:
from datasets import load_dataset

ds = load_dataset("cointegrated/taiga_stripped_rest", split='Fontanka')

In [ ]:
ds[0]

{'text': '«Газпром» и Белоруссия подписали соглашение о поставках российского газа.\nКак передает ИА «Регнум», глава компании Алексей Миллер сообщил журналистам подробности соглашения.\n\nЦена российского газа для Белоруссии составит с 1 января 100 долларов США за 1000 кубометров и в дальнейшем будет рассчитываться по формуле цены, установленной в контракте. К 2011 году она вырастет до среднеевропейского уровня. \n\nСтоимость транспортировки российского газа по территории Белоруссии вырастет с нынешних 0,75 доллара США за 1000 кубометров на 100 километров до 1,45 доллара США и будет зафиксирована на все пять лет действия контракта. \n\n"Газпром" в ближайшие 4 года выкупит 50% акций "Белтрансгаза" за 2,5 миллиарда долларов в течение 4 лет. \n\nПоследние переговоры между сторонами прошли вчера в Москве.\n                ',
 'file': 'Fontanka/texts/2007/fontanka_20070101001.txt'}

In [ ]:
fontanka_data = []
for elem in ds:
    fontanka_data.append(elem['text'])
fontanka_data = '\n'.join(fontanka_data)

In [ ]:
fontanka_data = fontanka_data[:40000000]

In [ ]:
def normalize(text):
    normalized_text = [word.text for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [ ]:
sentences_fontanka = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(fontanka_data)]

In [ ]:
train = sentences_fontanka[:-50]

In [ ]:
test = sentences_fontanka[-50:]

In [ ]:
len(sentences_fontanka)

322861

In [ ]:
len(train)

322811

In [ ]:
len(test)

50

In [ ]:
from scipy.sparse import lil_matrix

In [ ]:
train[0]

['<start>',
 '<start>',
 '«',
 'газпром',
 '»',
 'и',
 'белоруссия',
 'подписали',
 'соглашение',
 'о',
 'поставках',
 'российского',
 'газа',
 '.',
 '<end>']

In [ ]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sent in train:
    for i in range(len(sent)):
        unigrams[sent[i]] += 1
        if i>=1:
            bigrams[(sent[i-1], sent[i])] += 1
        if i>=2:
            trigrams[(sent[i-2], sent[i-1], sent[i])] += 1

In [ ]:
id2word = list(unigrams)
word2id = {word:i for i, word in enumerate(id2word)}
id2bigram = list(bigrams)
bigram2id = {bg:i for i,bg in enumerate(id2bigram)}

$P(w3​∣w1​,w2​)=\frac{count(w1​,w2​)}{count(w1​,w2​,w3​)}​$

In [ ]:
# основная матрица: биграммы на униграммы
matrix = lil_matrix((len(bigrams), len(unigrams)))
for (w1, w2, w3), count in trigrams.items():
    row = bigram2id[(w1, w2)]
    col = word2id[w3]
    matrix[row, col] = count / bigrams[(w1, w2)]
matrix[0,5]

np.float64(9.29336360904678e-06)

In [ ]:
# матрица: униграммы на униграммы
bigram_matrix = lil_matrix((len(unigrams), len(unigrams)))
for (w1, w2), count in bigrams.items():
    i, j = word2id[w1], word2id[w2]
    bigram_matrix[i,j] = count / unigrams[w1]

In [ ]:
# вероятность просто отдельного слова
count_all_words = sum(unigrams.values())
unigram_probs = np.array([unigrams[w] / count_all_words for w in id2word], dtype=float)
unigram_probs

array([8.65206586e-02, 1.43095951e-02, 9.17977565e-05, ...,
       1.34011323e-07, 1.34011323e-07, 1.34011323e-07])

In [ ]:
def apply_temperature(probas, temperature):
    log_probas = np.log(np.maximum(probas, 1e-10))
    adjusted_log_probas = log_probas / temperature
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

In [ ]:
def generate(matrix, id2bigram, id2word, bigram2id, word2id, bigram_matrix, unigram_probs,
            n=600, temperature=0.9, backoff_threshold=1e-10):
    w1, w2 = '<start>', '<start>'
    text = []
    sentence = []
    for i in range(n):
        row_idx = bigram2id.get((w1, w2))
        probs = None
        if row_idx is not None:
            probs = matrix[row_idx].toarray()[0]
            # если для данной биграммы совсем маленькая вероятность
            # то переходим к биграмной матрице
            if probs.max() < backoff_threshold:
                probs = None
        if probs is None:
            idx_w2 = word2id.get(w2)
            if idx_w2 is not None:
                probs = bigram_matrix[idx_w2].toarray()[0]
            else:
                row_idx = bigram2id.get(('<start>', w2))
                if row_idx is None:
                    probs = unigram_probs

        chosen_idx = np.random.choice(matrix.shape[1], p=apply_temperature(probs, temperature=temperature))
        chosen_word = id2word[chosen_idx]
        if chosen_word == '.' and len(sentence) <= 3:
            breaking_point = 0
            while True:
                if breaking_point == 100:
                    break
                chosen_idx = np.random.choice(matrix.shape[1],
                                                  p=apply_temperature(probs, temperature=0.9))
                chosen_word = id2word[chosen_idx]
                if chosen_word != '.':
                    break
                breaking_point += 1

        if chosen_word == '<end>':
            final_sentence = ' '.join(sentence).capitalize()
            text.append(f'{final_sentence}<end>')
            sentence = []
            if n - i < 10:
                break
        else:
            sentence.append(chosen_word)
        w1, w2 = w2, chosen_word
    return ' '.join(text)

In [ ]:
texts = []
for _ in range(5):
    texts.append(generate(matrix, id2bigram, id2word, bigram2id, word2id,
               bigram_matrix, unigram_probs, temperature=0.65, n=100).replace('<end>', ' '))

In [ ]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [ ]:
def compute_join_proba_trigrams(seq, unigram_counts, bigram_counts, trigram_counts):
    log_prob = 0.0
    for i in range(2, len(seq)):
        w1, w2, w3 = seq[i-2], seq[i-1], seq[i]

        bigram = (w1, w2)
        trigram = (w1, w2, w3)

        if bigram in bigram_counts and trigram in trigram_counts:
            p = trigram_counts[trigram] / bigram_counts[bigram]
            log_prob += np.log(p)
        else:
            log_prob += np.log(2e-6)

    return log_prob, len(seq[2:-1])

In [ ]:
def compute_joint_proba(tokens, word_probas):
    prob = 0
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)

    return prob, len(tokens[2:-1])


def compute_join_proba_markov_assumption(tokens, word_counts, bigram_counts):
    prob = 0
    for i in range(1, len(tokens)):
        w1, w2 = tokens[i-1], tokens[i]
        if w1 in word_counts and w2 in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens[2:-1])

In [ ]:
all_results_trigrams = []
all_results_bigrams = []
all_results_uni = []
for phrase in test:
    log_prob, tokens_length = compute_join_proba_trigrams(phrase, unigrams, bigrams, trigrams)
    all_results_trigrams.append(perplexity(log_prob, tokens_length))
    log_prob, tokens_length = compute_join_proba_markov_assumption(phrase, unigrams, bigrams)
    all_results_bigrams.append(perplexity(log_prob, tokens_length))
    log_prob, tokens_length = compute_joint_proba(phrase, unigram_probs)
    all_results_uni.append(perplexity(log_prob, tokens_length))

In [ ]:
print(f'По триграммам перплексия: {np.mean(all_results_trigrams)}')
print(f'По биграммам перплексия: {np.mean(all_results_bigrams)}')
print(f'По униграммам перплексия: {np.mean(all_results_uni)}')

По триграммам перплексия: 32236.708598509824
По биграммам перплексия: 218078.45827201704
По униграммам перплексия: 30294.50888753311


In [ ]:
import re

In [ ]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def normalize_texts(text):
    text = re.sub(' (?=[!#&*+,.»/:;?]+)', '', text)
    text = re.sub('(?<=«) ', '', text)
    return text

In [ ]:
print(f'{'◦'*50} Полученные тексты {'◦'*50}')
for text in texts:
    print(normalize_texts(text))
    print()

◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦ Полученные тексты ◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦◦
В приморском районе стреляли в легковую машину риа «новости».  Adsl-модем всем своим видом давал понять присутствующим, что в течение того же года с инсталляцией пра-образы в art-галерее 103 на пушкинской улице вернулось в московский городской суд санкт-петербурга отказал в иске к «».  Перевыполнить!  Сочиненную одной из крупнейших музейных комплексов, то есть в качестве официального периодического издания, которые в принципе, все с простого человека, который будет способствовать развитию этой компании. 

Нам же известно и о странных и злых людях, которые они оказывали ему медпомощь.  Шлем, в городе на неве, а другие - пятизвездочный отель etc, ориентирована отнюдь не на “ лакомый кусок ” в конце апреля, и в метро?  Безотлагательные меры по предупреждению и ликвидации последствий аварии: вторая в городе и ленобласти, это была целая эпопея, - заявила «фонтанке» в пресс

## Задание № 2 (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [ ]:
class Beam:
    def __init__(self, sequence, score):
        self.sequence = sequence
        self.score = score

def generate_with_beam_search_trigram(trigram_matrix, id2word, word2id, bigram2id,
                                      n=50, max_beams=5, prompt=None):

    if prompt is None:
        seq = ['<start>', '<start>']
    else:
        tokens = prompt.split()
        tokens = ['<start>', '<start>'] + tokens
        seq = tokens[-2:]  # последние два слова только

    beams = [Beam(sequence=seq, score=0.0)]

    for step in range(n):
        new_beams = []
        for beam in beams:
            w1, w2 = beam.sequence[-2], beam.sequence[-1]
            if w2 == '<end>':
                new_beams.append(beam)
                continue

            bigram_id = bigram2id.get((w1, w2))

            if bigram_id is None:
                continue

            probs = trigram_matrix[bigram_id].toarray()[0]
            top_ids = probs.argsort()[:-(max_beams+1):-1]

            for wid in top_ids:
                p = probs[wid]
                if p <= 0:
                    continue

                new_seq = beam.sequence + [id2word[wid]]
                new_score = beam.score + np.log(p)

                new_beams.append(Beam(new_seq, new_score))

        beams = sorted(new_beams, key=lambda b: b.score, reverse=True)[:max_beams]

        if all(b.sequence[-1] == '<end>' for b in beams):
            break

    best = sorted(beams, key=lambda b: b.score, reverse=True)
    return [" ".join(b.sequence) for b in best]


In [ ]:
results = generate_with_beam_search_trigram(
    trigram_matrix=matrix,
    id2word=id2word,
    word2id=word2id,
    bigram2id=bigram2id,
    n=150,
    max_beams=10
)

for r in results:
    print(r)

<start> <start> как передает корреспондент « фонтанки » . <end>
<start> <start> как сообщили корреспонденту « фонтанки » . <end>
<start> <start> как передает « газета . ru » . <end>
<start> <start> как передает корреспондент « фонтанки » , - сказал он . <end>
<start> <start> как передает корреспондент « фонтанки » в пресс-службе гу мчс рф по петербургу и ленобласти . <end>
<start> <start> как передает « газета . ru » со ссылкой на пресс-службу гу мчс рф по петербургу и ленобласти . <end>
<start> <start> как передает « газета . ru » со ссылкой на пресс-службу гу мчс рф по петербургу и ленинградской области , в том , что в петербурге . <end>
<start> <start> как передает « газета . ru » со ссылкой на пресс-службу гу мчс рф по петербургу и ленинградской области , в том , что в ближайшее время . <end>
<start> <start> как передает « газета . ru » со ссылкой на пресс-службу гу мчс рф по петербургу и ленинградской области , в том , что в случае , если бы не было . <end>
<start> <start> как пер

Без промпта и с бим-серч, результаты более детерминиированные и ориентируется только на один шаблон новостных текстов. Они все начинаются с ссылки на другой новостной источник.

In [ ]:
results = generate_with_beam_search_trigram(
    matrix, id2word, word2id, bigram2id,
    n=150,
    max_beams=10,
    prompt="согласно сообщению"
)

In [ ]:
for r in results:
    print(r)

согласно сообщению пресс-службы обеих сторон . <end>
согласно сообщению пресс-службы жилищного комитета юнис лукманов . <end>
согласно сообщению пресс-службы кбдх дальнейший график закрытия будет корректироваться . <end>
согласно сообщению пресс-службы жилищного комитета юниса лукманова . <end>
согласно сообщению пресс-службы кбдх дальнейший график закрытия движения от стадиона « петровский » . <end>
согласно сообщению пресс-службы кбдх дальнейший график закрытия движения от стадиона « петровский » в пресс-службе гу мчс рф по петербургу и ленобласти . <end>
согласно сообщению пресс-службы кбдх дальнейший график закрытия движения от стадиона « петровский » в пресс-службе гу мчс рф по петербургу и ленинградской области , в том , что в петербурге . <end>
согласно сообщению пресс-службы кбдх дальнейший график закрытия движения от стадиона « петровский » в пресс-службе гу мчс рф по петербургу и ленинградской области , в том , что в ближайшее время . <end>
согласно сообщению пресс-службы кбд

In [ ]:
results = generate_with_beam_search_trigram(
    matrix, id2word, word2id, bigram2id,
    n=150,
    max_beams=10,
    prompt="в районе"
)
for r in results:
    print(r)

в районе аварии затруднено . <end>
в районе аварии образовалась огромная пробка . <end>
в районе стадиона « петровский » . <end>
в районе площади восстания . <end>
в районе аварии образовалась пробка . <end>
в районе станции метро « ладожская » . <end>
в районе площади восстания ; фоторепортаж с думской улицы . <end>
в районе площади восстания ; фоторепортаж с площади восстания . <end>
в районе площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с думской улицы . <end>
в районе площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади восстания ; фоторепортаж с площади вос

In [ ]:
results = generate_with_beam_search_trigram(
    matrix, id2word, word2id, bigram2id,
    n=150,
    max_beams=10,
    prompt="по предварительным данным"
)
for r in results:
    print(r)

предварительным данным , причиной аварии стало заложенное на путях . <end>
предварительным данным , причиной аварии стало заложенное на путях пожарной эвакуации . <end>
предварительным данным , причиной аварии стало заложенное на путях самодельное взрывное устройство . <end>
предварительным данным , причиной аварии стало заложенное на рельсах лежит человек . <end>
предварительным данным , причиной аварии стало заложенное на путях самодельное взрывное устройство мощностью около двух килограммов тротила . <end>
предварительным данным , причиной аварии стало заложенное на путях самодельное взрывное устройство мощностью около двух часов . <end>
предварительным данным , причиной аварии стало заложенное на путях самодельное взрывное устройство мощностью около двух часов дня . <end>
предварительным данным , причиной аварии стало заложенное на путях самодельное взрывное устройство мощностью около двух лет . <end>
предварительным данным , причиной аварии стало заложенное на путях самодельное вз

In [ ]:
results = generate_with_beam_search_trigram(
    matrix, id2word, word2id, bigram2id,
    n=150,
    max_beams=10,
    prompt="они заявили"
)
for r in results:
    print(r)

они заявили на пресс-конференции в москве . <end>
они заявили на сегодняшнем заседании городского правительства . <end>
они заявили на сегодняшнем заседании городского парламента . <end>
они заявили на пресс-конференции в москве и петербурге . <end>
они заявили на сегодняшнем заседании городского штаба по благоустройству и дорожному хозяйству . <end>
они заявили на сегодняшнем заседании городского правительства губернатор валентина матвиенко . <end>
они заявили на сегодняшнем заседании городского правительства губернатор петербурга валентина матвиенко . <end>
они заявили на сегодняшнем заседании городского штаба по благоустройству и дорожному хозяйству , градостроительству и архитектуре . <end>
они заявили на сегодняшнем заседании городского штаба по благоустройству и дорожному хозяйству , градостроительству и архитектуре ( кга ) александр брод . <end>
они заявили на сегодняшнем заседании городского штаба по благоустройству и дорожному хозяйству , градостроительству и архитектуре ( кга

Проблема генераций, что точка как будто имеет довольно большую вероятность. Но одновременно отказываться от знаков препинания – грустно, потому что у нас в текстах появляются очень нечеловеческие тексты.